## Newspaper Churn
### Model Serving

In [16]:
import pandas as pd

# Features verification
from feature_engine.preprocessing import MatchVariables
# Categories verification
from feature_engine.preprocessing import MatchCategories
# Script Serving
from jpmml_evaluator import make_evaluator
from jpmml_evaluator.py4j import Py4JBackend
# REST Serving
from openscoring import Openscoring

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
# Data Import
df = pd.read_csv('../../../data/newspaper-churn-train.csv')
# Get sample dataset for model serving
df_serve = df.sample(30)
x_serve = df_serve.drop('target', axis=1)
y_serve = df_serve[['target']]
x_serve.reset_index(drop=True, inplace=True)
y_serve.reset_index(drop=True, inplace=True)

## Features Verification

In [19]:
# Ensures that columns in serving set match those in train set
x_train = df_serve.drop('target', axis=1)
# Setup the transformer
match_cols = MatchVariables(missing_values="ignore")
match_cols.fit(x_train)
match_cols.feature_names_in_

['HH Income',
 'Home Ownership',
 'Ethnicity',
 'dummy for Children',
 'Year Of Residence',
 'Age range',
 'Language',
 'City',
 'County',
 'Zip Code',
 'weekly fee',
 'Deliveryperiod',
 'Nielsen Prizm',
 'reward program',
 'Source Channel']

In [20]:
# Run the transformer
x_match = match_cols.transform(x_serve)
x_match.head(5)

,HH Income,Home Ownership,Ethnicity,dummy for Children,Year Of Residence,Age range,Language,City,County,Zip Code,weekly fee,Deliveryperiod,Nielsen Prizm,reward program,Source Channel
0,"$300,000 - $399,999",OWNER,English,N,20,75 years or more,English,HUNTINGTON BEACH,ORANGE,92648,$5.00 - $5.99,7Day,MW,3,CustCall
1,"$ 30,000 - $39,999",OWNER,English,N,28,65-69,English,ALISO VIEJO,ORANGE,92655,$4.00 - $4.99,7Day,MM,2,TeleIn
2,"Under $20,000",RENTER,Hispanic,N,5,70-74,Spanish,GARDEN GROVE,ORANGE,92840,$3.00 - $3.99,THU-SUN,FE,0,Crew1
3,"$ 90,000 - $99,999",OWNER,Italian,N,22,55-59,English,IRVINE,ORANGE,92605,$3.00 - $3.99,SunOnly,MW,0,Partner
4,"$100,000 - $124,999",OWNER,Indian,N,7,40-44,English,CHINO HILLS,SAN BERNARDINO,91709,$0.01 - $0.50,SunOnly,FW,0,Partner


## Categories Verification

In [21]:
# Ensures categorical variables are of type ‘category’
mc = MatchCategories(missing_values="ignore")
# mc.fit(x_train)

In [22]:
# Run the transformer
# x_match_c = mc.transform(x_match)
# x_match_c.head(5)

## Model Script Serving

In [29]:
# Load PMML model
evaluator = make_evaluator(backend=Py4JBackend(), path="../../../data/newspaper-churn-train-ensemble-classifier.pmml")
# Perform automated QA
evaluator.verify()

In [34]:
# Model Serving
dfresult = evaluator.evaluateAll(x_match)
dfresult.head(30)

,target,probability(0),probability(1)
0,1,0.125088,0.874912
1,1,0.354135,0.645865
2,1,0.125482,0.874518
3,1,0.391267,0.608733
4,1,0.276924,0.723076
5,1,0.446762,0.553238
6,1,0.471755,0.528245
7,1,0.370905,0.629095
8,0,0.520061,0.479939
9,0,0.708561,0.291439


## Model REST Serving

In [31]:
# Creating an Openscoring object
# Openscoring server have to be running on base-url
os = Openscoring(base_url = "http://localhost:8080", token="secret")

In [32]:
# Deploying a PMML model:
model_name = 'newspaper-churn-train-ensemble-classifier'
os.deployFile(model_name, "../../../data/newspaper-churn-train-ensemble-classifier.pmml")

In [33]:
# Model serving with data records from DataFrame
dfResponse = os.evaluateCsv(model_name, x_match)
print(dfResponse.head(5))

   target  probability(0)  probability(1)
0       1        0.125088        0.874912
1       1        0.354135        0.645865
2       1        0.125482        0.874518
3       1        0.391267        0.608733
4       1        0.276924        0.723076
